In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [7]:
print(y_test[0])

7


In [8]:
from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model 

In [18]:
class MNISTModel(Model):
    def __init__(self):
        super(MNISTModel, self).__init__(self)
        self.conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(128,activation ='relu')
        self.dense2 = Dense(10, activation ='softmax')
    
    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self. dense1(x2)
        return self.dense2(x3)
    
model = MNISTModel()

In [19]:
loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam()

In [20]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

In [21]:
test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [22]:
@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_function(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    train_loss(loss)
    train_accuracy(outputs,predictions)
    

In [23]:
@tf.function
def test_step(inputs, outputs):
    predictions = model(inputs)
    loss = loss_function(outputs, predictions)     
    train_loss(loss)
    train_accuracy(outputs,predictions)

In [26]:
x_train, x_test = x_train/255.0, x_test/255.0 
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [27]:
print(x_train[0])

[[[[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]]


 [[[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.        ]]

  [[0.  

In [28]:
train_data = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)


test_data = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)

In [30]:
epochs = 5 
for epoch in range(epochs):
    for train_inputs, train_labels in train_data:
        train_step(train_inputs, train_labels)
    
    for test_inputs, test_labels in test_data:
        test_step(test_inputs, test_labels)
        
    
    template = 'Epochs, {}, Train loss{}, Train Accuray: {} Tess loss: {} Test Accuracy: {}'
    print(template.format(
        epoch +1,
        train_loss.result(),
        train_accuracy.result(),
        test_loss.result(),
        test_accuracy.result(),
    ))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()
    

ValueError: in user code:

    File "/var/folders/jz/t5_njm3921b12l9h30d_c11r0000gn/T/ipykernel_33176/2332123277.py", line 4, in train_step  *
        predictions = model(inputs)
    File "/Users/vikashpirthiani/anaconda3/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
        raise e.with_traceback(filtered_tb) from None
    File "/var/folders/jz/t5_njm3921b12l9h30d_c11r0000gn/T/__autograph_generated_filea4el1b6w.py", line 10, in tf__call
        x1 = ag__.converted_call(ag__.ld(self).conv1, (ag__.ld(x),), None, fscope)

    ValueError: Exception encountered when calling layer 'mnist_model_3' (type MNISTModel).
    
    in user code:
    
        File "/var/folders/jz/t5_njm3921b12l9h30d_c11r0000gn/T/ipykernel_33176/4152066844.py", line 10, in call  *
            x1 = self.conv1(x)
        File "/Users/vikashpirthiani/anaconda3/lib/python3.11/site-packages/keras/utils/traceback_utils.py", line 70, in error_handler  **
            raise e.wit

In [32]:
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow.keras.layers import Conv2D, Flatten, Dense
from tensorflow.keras import Model 

mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

class MNISTModel(Model):
    def __init__(self):
        super(MNISTModel, self).__init__(self)
        self.conv1 = Conv2D(filters=32, kernel_size=3, activation='relu')
        self.flatten = Flatten()
        self.dense1 = Dense(128,activation ='relu')
        self.dense2 = Dense(10, activation ='softmax')
    
    def call(self, x):
        x1 = self.conv1(x)
        x2 = self.flatten(x1)
        x3 = self. dense1(x2)
        return self.dense2(x3)
    
model = MNISTModel()

loss_function = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.legacy.Adam()


train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

@tf.function
def train_step(inputs, outputs):
    with tf.GradientTape() as tape:
        predictions = model(inputs)
        loss = loss_function(outputs, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
        
    train_loss(loss)
    train_accuracy(outputs,predictions)

@tf.function
def test_step(inputs, outputs):
    predictions = model(inputs)
    loss = loss_function(outputs, predictions)     
    test_loss(loss)
    test_accuracy(outputs,predictions)

x_train, x_test = x_train/255.0, x_test/255.0 
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


train_data = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)


test_data = tf.data.Dataset.from_tensor_slices(
    (x_test, y_test)).batch(32)



epochs = 5 
for epoch in range(epochs):
    for train_inputs, train_labels in train_data:
        train_step(train_inputs, train_labels)
    
    for test_inputs, test_labels in test_data:
        test_step(test_inputs, test_labels)
        
    
    template = 'Epochs, {}, Train loss{}, Train Accuray: {} Tess loss: {} Test Accuracy: {}'
    print(template.format(
        epoch +1,
        train_loss.result(),
        train_accuracy.result(),
        test_loss.result(),
        test_accuracy.result(),
    ))
    
    train_loss.reset_states()
    train_accuracy.reset_states()
    test_loss.reset_states()
    test_accuracy.reset_states()


2024-01-13 22:52:46.160784: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype double and shape [60000,28,28,1]
	 [[{{node Placeholder/_0}}]]
2024-01-13 22:52:46.160909: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and shape [60000]
	 [[{{node Placeholder/_1}}]]
2024-01-13 22:52:53.296370: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_1' with dtype uint8 and sha

Epochs, 1, Train loss0.13871663808822632, Train Accuray: 0.9574499726295471 Tess loss: 0.05974791571497917 Test Accuracy: 0.9794999957084656
Epochs, 2, Train loss0.04510517045855522, Train Accuray: 0.9857833385467529 Tess loss: 0.05614371597766876 Test Accuracy: 0.982200026512146
Epochs, 3, Train loss0.025271551683545113, Train Accuray: 0.9915666580200195 Tess loss: 0.05200200900435448 Test Accuracy: 0.9840999841690063
Epochs, 4, Train loss0.015894971787929535, Train Accuray: 0.9945833086967468 Tess loss: 0.05816568806767464 Test Accuracy: 0.9825000166893005
Epochs, 5, Train loss0.010533778928220272, Train Accuray: 0.9962999820709229 Tess loss: 0.060483407229185104 Test Accuracy: 0.9835000038146973
